In [39]:
import pandas as pd
import numpy as np
import joblib
import os
import time
import datetime
from pytz import timezone
from datetime import timedelta
import configparser as cp
from sqlalchemy import create_engine
import mysql.connector
import yfinance as yf
file_path=os.getcwd()
file_path

'/home/ubuntu/projects/Stock_Price_Prediction'

# Database Connection

In [82]:
config=cp.ConfigParser()
config.read('/home/ubuntu/certi/db_login.txt')
db_config=config['ivan_db']
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'))

# Yahoo Finace

## 01 yfinance 
https://aroussi.com/post/python-yahoo-finance

### Testing 

In [3]:
import yfinance as yf

In [4]:
crm=yf.Ticker('CRM')
crm.history(period='max')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2004-06-23,3.750000,4.325000,3.687500,4.300000,43574400,0,0.0
2004-06-24,4.387500,4.422500,4.125000,4.190000,8887200,0,0.0
2004-06-25,4.127500,4.187500,3.947500,3.950000,6710000,0,0.0
2004-06-28,4.000000,4.052500,3.860000,4.000000,2270800,0,0.0
2004-06-29,4.000000,4.175000,3.957500,4.100000,2112000,0,0.0
...,...,...,...,...,...,...,...
2021-08-13,251.710007,253.770004,248.869995,251.559998,5985700,0,0.0
2021-08-16,250.169998,251.000000,244.470001,249.199997,4047500,0,0.0
2021-08-17,248.259995,249.199997,245.940002,246.990005,3472600,0,0.0


In [5]:
## data downloading
data=yf.download(tickers='CRM AAPL MSFT', # 'CRM AAPL MSFT'
                 periods='max',
                 interval='1d',
                 group_by='date',
                 auto_adjust=True,
                 prepost=False, # fownlaod pre/post regular market hours data
                 threads=True,
                 proxy=None)

[*********************100%***********************]  3 of 3 completed


In [6]:
pd.DataFrame(data).reset_index()

Date        MSFT                                                  \
                        Open        High         Low       Close      Volume   
0     1980-12-12         NaN         NaN         NaN         NaN         NaN   
1     1980-12-15         NaN         NaN         NaN         NaN         NaN   
2     1980-12-16         NaN         NaN         NaN         NaN         NaN   
3     1980-12-17         NaN         NaN         NaN         NaN         NaN   
4     1980-12-18         NaN         NaN         NaN         NaN         NaN   
...          ...         ...         ...         ...         ...         ...   
10254 2021-08-13  288.926880  292.340328  288.747201  292.290436  18231400.0   
10255 2021-08-16  292.629800  294.256690  289.465843  294.037109  22507600.0   
10256 2021-08-17  291.831335  292.869326  290.523810  292.519989  20075300.0   
10257 2021-08-18  292.040009  294.820007  290.269989  290.730011  21813000.0   
10258 2021-08-19  288.690002  297.470001  288.640015  296.769989  29821100.0   

             AAPL                                                        CRM  \
             Open        High         Low       Close     Volume        Open   
0        0.100600    0.101037    0.100600    0.100600  469033600         NaN   
1        0.095789    0.095789    0.095352    0.095352  175884800         NaN   
2        0.088790    0.088790    0.088353    0.088353  105728000         NaN   
3        0.090540    0.090977    0.090540    0.090540   86441600         NaN   
4        0.093165    0.093602    0.093165    0.093165   73449600         NaN   
...           ...         ...         ...         ...        ...         ...   
10254  148.970001  149.440002  148.270004  149.100006   59318800  251.710007   
10255  148.539993  151.190002  146.470001  151.119995  103296000  250.169998   
10256  150.229996  151.679993  149.089996  150.190002   92229700  248.259995   
10257  149.800003  150.720001  146.149994  146.360001   86326000  248.000000   
10258  145.029999  148.000000  144.500000  146.699997   86861700  249.169998   

                                                      
             High         Low       Close     Volume  
0             NaN         NaN         NaN        NaN  
1             NaN         NaN         NaN        NaN  
2             NaN         NaN         NaN        NaN  
3             NaN         NaN         NaN        NaN  
4             NaN         NaN         NaN        NaN  
...           ...         ...         ...        ...  
10254  253.770004  248.869995  251.559998  5985700.0  
10255  251.000000  244.470001  249.199997  4047500.0  
10256  249.199997  245.940002  246.990005  3472600.0  
10257  256.799988  247.720001  251.220001  8306700.0  
10258  255.440002  248.380005  253.070007  6598600.0  

[10259 rows x 16 columns]

### Programmatic 

In [7]:
col=pd.DataFrame()

for i in ['CRM','AAPL','MSFT']:
    df=yf.download(tickers='{}'.format(i),
                   periods='max',
                   interval='1d',
                   group_by='ticker',
                   auto_adjust=True,
                   prepost=False,
                   threads=True,
                   proxy=None
                  ).reset_index()
    df.loc[:,'Ticker']=i
    col=pd.concat([col,df],axis=0)
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
stock_list=[]

for i in ['CRM','AAPL','MSFT']:
    df=yf.download(tickers='{}'.format(i),
                   periods='max',
                   interval='1d',
                   group_by='ticker',
                   auto_adjust=True,
                   prepost=False,
                   threads=True,
                   proxy=None
                  ).reset_index()
    df['stock']=i
    stock_list.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
pd.concat(stock_list,axis=0)

,Date,Open,High,Low,Close,Volume,stock
0,2004-06-23,3.750000,4.325000,3.687500,4.300000,43574400,CRM
1,2004-06-24,4.387500,4.422500,4.125000,4.190000,8887200,CRM
2,2004-06-25,4.127500,4.187500,3.947500,3.950000,6710000,CRM
3,2004-06-28,4.000000,4.052500,3.860000,4.000000,2270800,CRM
4,2004-06-29,4.000000,4.175000,3.957500,4.100000,2112000,CRM
...,...,...,...,...,...,...,...
8928,2021-08-13,288.926880,292.340328,288.747201,292.290436,18231400,MSFT
8929,2021-08-16,292.629800,294.256690,289.465843,294.037109,22507600,MSFT
8930,2021-08-17,291.831335,292.869326,290.523810,292.519989,20075300,MSFT
8931,2021-08-18,292.040009,294.820007,290.269989,290.730011,21813000,MSFT


## Tickers
* https://www.nasdaq.com/market-activity/stocks/screener

### NYSE 

In [10]:
nyse=pd.read_csv(file_path+'/data/NYSE_Aug4_2021.csv')
nyse.columns=nyse.columns.str.replace(' ','').str.replace('%','')
## reformating
nyse.loc[:,'LastSale']=[float(x.replace('$',''))for x in nyse.LastSale]
nyse.loc[:,'Change']=[float(str(x).replace('%','')) for x in nyse['Change']]

print(nyse.shape)
print(nyse.Symbol.nunique())
nyse.head(5)

(3139, 11)
3139


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,154.27,0.41,0.266,4.681219e+10,United States,1999.0,1786735,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,38.75,-1.63,-4.037,7.241064e+09,NaN,2016.0,5081059,Basic Industries,Aluminum
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,9.72,-0.02,-0.205,1.215000e+09,NaN,2021.0,164241,Finance,Business Services
3,AAIC,Arlington Asset Investment Corp Class A (new),3.78,-0.03,-0.787,1.265589e+08,United States,NaN,162347,Finance,Finance/Investors Services
4,AAIC^B,Arlington Asset Investment Corp 7.00%,25.15,0.00,0.000,NaN,United States,NaN,52,NaN,NaN


#### Reviewing - NYSE

In [11]:
nyse.dtypes

Symbol        object
Name          object
LastSale     float64
NetChange    float64
Change       float64
MarketCap    float64
Country       object
IPOYear      float64
Volume         int64
Sector        object
Industry      object
dtype: object

In [12]:
print(nyse.loc[nyse.MarketCap<=100000,'Symbol'].nunique()) # $1 million
nyse.MarketCap.describe()

128


count    2.677000e+03
mean     1.361717e+10
std      4.044338e+10
min      0.000000e+00
25%      4.256438e+08
50%      1.869335e+09
75%      8.361171e+09
max      6.157856e+11
Name: MarketCap, dtype: float64

In [13]:
## NAs
pd.DataFrame({'vals':nyse.isnull().sum()}).loc[lambda x:x.vals>0,:]

,vals
Change,1
MarketCap,462
Country,627
IPOYear,1504
Sector,752
Industry,752


#### Filtering - NYSE

In [14]:
nyse_2=nyse.loc[nyse.MarketCap>=1000000,:].reset_index(drop=True)
nyse_2.loc[:,'SE']='NYSE'
print(nyse_2.shape)
nyse_2.head(5)

(2549, 12)


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry,SE
0,A,Agilent Technologies Inc. Common Stock,154.27,0.41,0.266,4.681219e+10,United States,1999.0,1786735,Capital Goods,Electrical Products,NYSE
1,AA,Alcoa Corporation Common Stock,38.75,-1.63,-4.037,7.241064e+09,NaN,2016.0,5081059,Basic Industries,Aluminum,NYSE
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,9.72,-0.02,-0.205,1.215000e+09,NaN,2021.0,164241,Finance,Business Services,NYSE
3,AAIC,Arlington Asset Investment Corp Class A (new),3.78,-0.03,-0.787,1.265589e+08,United States,NaN,162347,Finance,Finance/Investors Services,NYSE
4,AAN,Aarons Holdings Company Inc. Common Stock,27.61,-1.59,-5.445,9.020895e+08,United States,2020.0,168536,Miscellaneous,Diversified Commercial Services,NYSE


In [15]:
nyse_2.to_csv(file_path+'/data/NYSE_Aug4_2021_filtered.csv',index=False)

### NASDAQ 

In [16]:
nasdaq=pd.read_csv(file_path+'/data/nasdaq_Aug4_2021.csv')
nasdaq.columns=nasdaq.columns.str.replace(' ','').str.replace('%','')
## reformating
nasdaq.loc[:,'LastSale']=[float(x.replace('$','')) for x in nasdaq.LastSale]
nasdaq.loc[:,'Change']=[float(str(x).replace('%','')) for x in nasdaq.Change]


print(nasdaq.shape)
print(nasdaq.Symbol.nunique())
nasdaq.head(5)

(4417, 11)
4417


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,2.90,0.02,0.694,9.193344e+07,China,NaN,25944,Miscellaneous,Service to the Health Industry
1,AAL,American Airlines Group Inc. Common Stock,19.45,-0.50,-2.506,1.259305e+10,United States,NaN,30665670,Transportation,Air Freight/Delivery Services
2,AAME,Atlantic American Corporation Common Stock,4.20,-0.05,-1.176,8.574628e+07,United States,NaN,21993,Finance,Life Insurance
3,AAOI,Applied Optoelectronics Inc. Common Stock,7.71,0.09,1.181,2.071067e+08,United States,2013.0,563275,Technology,Semiconductors
4,AAON,AAON Inc. Common Stock,62.08,-0.90,-1.429,3.255175e+09,United States,NaN,94138,Capital Goods,Industrial Machinery/Components


#### Reviewing - Nasdaq

In [17]:
nasdaq.dtypes

Symbol        object
Name          object
LastSale     float64
NetChange    float64
Change       float64
MarketCap    float64
Country       object
IPOYear      float64
Volume         int64
Sector        object
Industry      object
dtype: object

In [18]:
print(nyse.loc[nasdaq.MarketCap<=100000,'Symbol'].nunique()) # $1 million
nasdaq.MarketCap.describe()

703


count    4.402000e+03
mean     6.340271e+09
std      7.303292e+10
min      0.000000e+00
25%      3.342577e+07
50%      2.633665e+08
75%      1.224359e+09
max      2.547722e+12
Name: MarketCap, dtype: float64

In [19]:
pd.DataFrame({'vals':nasdaq.isnull().sum()}).loc[lambda x:x.vals>0]

,vals
Change,7
MarketCap,15
Country,7
IPOYear,1626
Sector,858
Industry,859


#### Filtering - Nasdaq

In [20]:
nasdaq_2=nasdaq.loc[nasdaq.MarketCap>=1000000,:].reset_index(drop=True)
nasdaq_2.loc[:,'SE']='NASDAQ'
print(nasdaq_2.shape)
nasdaq_2.head(5)

(3442, 12)


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry,SE
0,AACG,ATA Creativity Global American Depositary Shares,2.90,0.02,0.694,9.193344e+07,China,NaN,25944,Miscellaneous,Service to the Health Industry,NASDAQ
1,AAL,American Airlines Group Inc. Common Stock,19.45,-0.50,-2.506,1.259305e+10,United States,NaN,30665670,Transportation,Air Freight/Delivery Services,NASDAQ
2,AAME,Atlantic American Corporation Common Stock,4.20,-0.05,-1.176,8.574628e+07,United States,NaN,21993,Finance,Life Insurance,NASDAQ
3,AAOI,Applied Optoelectronics Inc. Common Stock,7.71,0.09,1.181,2.071067e+08,United States,2013.0,563275,Technology,Semiconductors,NASDAQ
4,AAON,AAON Inc. Common Stock,62.08,-0.90,-1.429,3.255175e+09,United States,NaN,94138,Capital Goods,Industrial Machinery/Components,NASDAQ


In [21]:
## Svaing
nasdaq_2.to_csv(file_path+'/data/nasdaq_Aug4_2021_filtered.csv',index=False)

### Merge 

In [22]:
all_tickers=pd.concat([nyse_2,nasdaq_2],axis=0).reset_index(drop=True)
all_tickers.to_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv',index=False)
print(all_tickers.shape)
print(all_tickers.Symbol.nunique())
all_tickers.head(5)

(5991, 12)
5991


,Symbol,Name,LastSale,NetChange,Change,MarketCap,Country,IPOYear,Volume,Sector,Industry,SE
0,A,Agilent Technologies Inc. Common Stock,154.27,0.41,0.266,4.681219e+10,United States,1999.0,1786735,Capital Goods,Electrical Products,NYSE
1,AA,Alcoa Corporation Common Stock,38.75,-1.63,-4.037,7.241064e+09,NaN,2016.0,5081059,Basic Industries,Aluminum,NYSE
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,9.72,-0.02,-0.205,1.215000e+09,NaN,2021.0,164241,Finance,Business Services,NYSE
3,AAIC,Arlington Asset Investment Corp Class A (new),3.78,-0.03,-0.787,1.265589e+08,United States,NaN,162347,Finance,Finance/Investors Services,NYSE
4,AAN,Aarons Holdings Company Inc. Common Stock,27.61,-1.59,-5.445,9.020895e+08,United States,2020.0,168536,Miscellaneous,Diversified Commercial Services,NYSE


In [23]:
all_tickers=pd.read_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv')

In [24]:
## Reviewing
all_tickers.SE.value_counts()

NASDAQ    3442
NYSE      2549
Name: SE, dtype: int64

In [25]:
## saving
all_tickers.to_sql(name='NYSE_NASDAQ_TICKERS',
                   con=engine,
                   if_exists='replace',
                   index=False,
                   chunksize=1000)

## Loop 

In [26]:
stock_list=[]


for i in nasdaq_2.Symbol.tolist():
    df=yf.download(tickers='{}'.format(i),
                   periods='max',
                   interval='1d',
                   group_by='ticker',
                   auto_adjust=True,
                   prepost=False,
                   threads=True,
                   proxy=None
                  ).reset_index()
    df['stock']=i
    stock_list.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:
datetime.datetime.now()

In [ ]:
## All
start_time=datetime.datetime.now()
stock_list=[]
for s in ['NASDAQ','NYSE']:
    count=1
    for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].tolist():
        if count%50==0:
            time.sleep(3)
            print('Wait every 50 queries: progress:{:.2f}%'.format(count/all_tickers.shape[0]*100))
        df=yf.download(tickers=i,
                       periods='max',
                       interval='1d',
                       groupby='ticker',
                       auto_adjust=True,
                       prepose=False,
                       threads=True,
                       proxy=None
                      ).reset_index()
        df['SE']=s
        df['Stock']=i
        stock_list.append(df)
        
        count+=1
        
print(datetime.datetime.now()-start_time)       

### Data Output 

In [ ]:
## 
hist_stock_price=pd.concat(stock_list)
joblib.dump(hist_stock_price,'/home/ubuntu/projects/Stock_Price_Prediction/data/stock_hist_Aug11.pkl')
print(hist_stock_price.shape)
hist_stock_price.groupby('SE').agg(stock_count=('Stock','nunique'))

In [ ]:
print()

In [81]:
file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv'

'/home/ubuntu/projects/Stock_Price_Prediction/data/NYSE_NASDAQ_Aug4_2021.csv'

In [7]:
def fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals=None,n_sample=None):
    stock_list=[]
    ## Loading tickers ##
    all_tickers=pd.read_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv')
    
    ## params ##
    if start_vals==None:
        period_vals='max'
    else:
        period_vals=None
    if n_sample==None:
        all_tickers=all_tickers
    else:
        all_tickers=all_tickers.sample(n=n_sample)
    
    ## fetching data ##
    count=1
    
    for s in stock_exchange:
        for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].tolist():
            if count%50==0:
                time.sleep(3)
                print('Wait every 50 queries; Progress: {:.2f}%'.format(count/all_tickers.shape[0]*100))
            
            df=yf.download(tickers=i,
                           periods=period_vals,
                           start=start_vals,
                           interval='1d',
                           groupby='ticker',
                           auto_adjust=True,
                           prepose=False,
                           threads=True,
                           proxy=None
                          ).reset_index()
            df['SE']=s
            df['Stock']=i
            stock_list.append(df)
            
            count+=1
    return pd.concat(stock_list)
    

In [78]:
fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals='2021-09-01',n_sample=10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DKDCA: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume,SE,Stock,Adj Close
0,2021-08-31,35.250000,35.490002,33.130001,33.889999,411284.0,NASDAQ,HRMY,NaN
0,2021-08-31,23.270000,23.270000,22.930000,23.100000,17838.0,NASDAQ,ESCA,NaN
0,2021-08-31,47.500000,48.570000,47.020000,48.290001,1729324.0,NASDAQ,HTHT,NaN
0,2021-08-31,12.170000,13.020000,11.710100,12.980000,90832.0,NASDAQ,FHTX,NaN
0,2021-08-31,20.180000,20.469999,20.100000,20.260000,791196.0,NYSE,KRG,NaN
0,2021-08-31,7.310000,7.800000,7.269200,7.730000,1229005.0,NYSE,BODY,NaN
0,2021-08-31,225.460007,226.080002,221.770004,222.910004,2425331.0,NYSE,DG,NaN
0,2021-08-31,35.459999,35.700001,35.305000,35.410000,1053421.0,NYSE,OGE,NaN
0,2021-08-31,9.900000,9.900000,9.880000,9.890000,64151.0,NYSE,ATHN,NaN


In [4]:
all_stock_hist=fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'])

NameError: name 'yf' is not defined

In [ ]:
print(all_stock_hist.shape)

In [ ]:
# all_stock_hist.to_csv('/home/ubuntu/projects/Stock_Price_Prediction/data/stock_hist_Aug11.csv',index=False)

### Get the latest data only 

In [ ]:
hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
hist_df

In [ ]:
start_dt=hist_df.loc[0,'Date']+timedelta(days=1)
start_dt

In [ ]:
fetch_stockprice_all(start_vals=start_dt)

In [ ]:
from airflow import MySQLHook

In [31]:
def update_all_stocks():
    # input new data into temp 
    # do minus
    # insert new data
    ## db connection
    config=cp.ConfigParser()
    config.read('/home/ubuntu/certi/db_login.txt')
    db_config=config['ivan_db']
    # conn -1
    engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                                   db_config['pwd'],
                                                                                   db_config['hostname'],
                                                                                 'STOCK_PRED'))
    # conn -2
    mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
                                 user=db_config['userid'],
                                 password=db_config['pwd'],
                                 database='STOCK_PRED')
    cursor=mydb.cursor()
    
    ## Get the start date:
    hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
    hist_last_day=hist_df.loc[0,'Date']
    print('Latest data in the database: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
    
    ## fetch the latest data
    latest_data=fetch_stockprice_all(start_vals=hist_last_day+timedelta(days=1))
    latest_data.loc[:,'REFRESH_DATE']=datetime.datetime.now()
    latest_data=latest_data.loc[:,['Date','Open','High','Low','Close','Volume','SE','Stock','REFRESH_DATE']]
    # Additional checking
    latest_data_2=latest_data.loc[latest_data.Date>hist_last_day,:].reset_index(drop=True)
    
    
    ## Load to temp table
    latest_data_2.to_sql(name='ALL_STOCK_LATEST',
                         con=engine,
                         if_exists='replace',
                         index=False,
                         chunksize=1000)
    print('ALL_STOCK_LATEST is updated with data as of {:s}'.format(latest_data.Date.max().strftime(format='%Y-%m-%d')))
    
    ## Comparing -1 
    #df_duplicate=curor.execute("""SELECT Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE,COUNT(*)
    #                              FROM (SELECT *
    #                                    FROM ALL_STOCK_HIST 
    #                                    UNION ALL
    #                                    SELECT *
    #                                    FROM ALL_STOCK_LATEST)
    #                              GROUP BY Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE
    #                              HAVING COUNT(*)>1""")
    #if df_duplicate.shape[0]==0:
    #    cursor.execute("""INSERT INTO ALL_STOCK_HIST
    #                      SELECT *
    #                      FROM ALL_STOCK_LATEST""")
    #    print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data.Date.max().strftime(format='%Y-%m-%d')))
    #    
    #else: 
    #    print('ALL_STOCK_HIST is already up-to-date')
    
    ## Comparing -2
    if hist_last_day<latest_data.Date.max():
        cursor.execute("""INSERT INTO ALL_STOCK_HIST
                          SELECT *
                          FROM ALL_STOCK_LATEST""")
        print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data.Date.max().strftime(format='%Y-%m-%d')))
    else:
        print('ALL_STOCK_HIST is already up-to-date, latest data: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
        

In [32]:
update_all_stocks()

Latest data in the database: 2021-08-13
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [73]:
def update_all_stocks_2():
    
    utc_tz=datetime.datetime.now()
    # input new data into temp 
    # do minus
    # insert new data
    ## db connection
    config=cp.ConfigParser()
    config.read('/home/ubuntu/certi/db_login.txt')
    db_config=config['ivan_db']
    # conn -1
    engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                                   db_config['pwd'],
                                                                                   db_config['hostname'],
                                                                                 'STOCK_PRED'))
    # conn -2
    mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
                                 user=db_config['userid'],
                                 password=db_config['pwd'],
                                 database='STOCK_PRED')
    cursor=mydb.cursor()
    
    ## Get the start date:
    hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
    hist_last_day=hist_df.loc[0,'Date']
    print('Latest data in the database: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
    
    if utc_tz.astimezone(timezone('America/New_York')).date()>hist_last_day:
        if utc_tz.astimezone(timezone('America/New_York')).weekday()+1<=5:
            ## fetch the latest data
            latest_data=fetch_stockprice_all(start_vals=hist_last_day+timedelta(days=1))
            
            if latest_data.shape[0]>0:
                latest_data.loc[:,'REFRESH_DATE']=datetime.datetime.now()
                latest_data=latest_data.loc[:,['Date','Open','High','Low','Close','Volume','SE','Stock','REFRESH_DATE']]
        
                # additional checking
                latest_data_2=latest_data.loc[latest_data.Date>hist_last_day,:].reset_index(drop=True)
    
                ## Load to temp table 
                latest_data_2.to_sql(name='ALL_STOCK_LATEST', # ALL_STOCK_LATEST
                                     con=engine,
                                     if_exists='replace',
                                     index=False,
                                     chunksize=1000)
                print('ALL_STOCK_LATEST is updated with data as of {:s}, rows updated: {:,.0f}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d'),
                                                                                                       latest_data_2.shape[0])
                     )
    
                cursor.execute("""INSERT INTO ALL_STOCK_HIST 
                                  SELECT *
                                  FROM ALL_STOCK_LATEST 
                                  COMMIT
                                """)
                mydb.commit()
                print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d')))
            
            else:
                print('Stock data not available')
        
        else:
            print('Not weekday')
    
    else:
        print('ALL_STOCK_HIST is already up-to-date, latest data: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
        
    return datetime.datetime.now()-utc_tz
        

In [74]:
update_all_stocks_2()

Latest data in the database: 2021-08-13
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Wait every 50 queries; Progress: 92.64%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Wait every 50 queries; Progress: 98.48%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 comp

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [76]:
hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
hist_last_day=hist_df.loc[0,'Date']
hist_last_day

Timestamp('2021-08-11 00:00:00')

In [80]:
datetime.datetime(2021,9,1)

datetime.datetime(2021, 9, 1, 0, 0)

# Airflow Dag

In [ ]:
from airflow import DAG
from datetime import datetime,timedelta
from airflow.operators.python import PythonOperator

## Projects
import pandas as pd
import numpy as np
import joblib
import os
import time
import datetime
from pytz import timezone
import configparser as cp
from sqlalchemy import create_engine
import mysql.connector
import yfinance as yf


## Airflow setup
yf_args={'owner':'airflow',
         'depends_on_past':False,
         'start_date':datetime(2021,9,1),
         'retries':3,
         'retry_delay':timedelta(minutes=5),
         'catchup':False # not in the documentation
        }


## db connection ##
config=cp.ConfigParser()
config.read('/home/ubuntu/certi/db_login.txt')
db_config=config['ivan_db']
# conn -1
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'))
# conn -2
mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
                             user=db_config['userid'],
                             password=db_config['pwd'],
                             database='STOCK_PRED')
cursor=mydb.cursor()

###############
## Functions 
###############
def fetch_stockprice_all(stock_exchange=['NASDAQ','NYSE'],start_vals=None,n_sample=None):
    stock_list=[]
    ## Loading tickers ##
    all_tickers=pd.read_csv(file_path+'/data/NYSE_NASDAQ_Aug4_2021.csv')
    
    ## params ##
    if start_vals==None:
        period_vals='max'
    else:
        period_vals=None
    if n_sample==None:
        all_tickers=all_tickers
    else:
        all_tickers=all_tickers.sample(n=n_sample)
    
    ## fetching data ##
    count=1
    
    for s in stock_exchange:
        for i in all_tickers.loc[all_tickers.SE==s,'Symbol'].tolist():
            if count%50==0:
                time.sleep(3)
                print('Wait every 50 queries; Progress: {:.2f}%'.format(count/all_tickers.shape[0]*100))
            
            df=yf.download(tickers=i,
                           periods=period_vals,
                           start=start_vals,
                           interval='1d',
                           groupby='ticker',
                           auto_adjust=True,
                           prepose=False,
                           threads=True,
                           proxy=None
                          ).reset_index()
            df['SE']=s
            df['Stock']=i
            stock_list.append(df)
            
            count+=1
    return pd.concat(stock_list)


###############
## Major 
###############
def update_all_stocks_2():
    
    utc_tz=datetime.datetime.now()
    # input new data into temp 
    # do minus
    # insert new data
    ## db connection
#    config=cp.ConfigParser()
#    config.read('/home/ubuntu/certi/db_login.txt')
#    db_config=config['ivan_db']
    # conn -1
#    engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
#                                                                                   db_config['pwd'],
#                                                                                   db_config['hostname'],
#                                                                                 'STOCK_PRED'))
    # conn -2
#    mydb=mysql.connector.connect(host=db_config['hostname'].replace(':3306',''),
#                                 user=db_config['userid'],
#                                 password=db_config['pwd'],
#                                 database='STOCK_PRED')
#    cursor=mydb.cursor()
    
    ## Get the start date:
    hist_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST ORDER BY Date DESC LIMIT 1", con=engine)
    hist_last_day=hist_df.loc[0,'Date']
    print('Latest data in the database: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
    
    if utc_tz.astimezone(timezone('America/New_York')).date()>hist_last_day:
        if utc_tz.astimezone(timezone('America/New_York')).weekday()+1<=5:
            ## fetch the latest data
            latest_data=fetch_stockprice_all(start_vals=hist_last_day+timedelta(days=1))
            
            if (latest_data.shape[0]>0) & (latest_data.Open.isnull().sum()==0):
                latest_data.loc[:,'REFRESH_DATE']=datetime.datetime.now()
                latest_data=latest_data.loc[:,['Date','Open','High','Low','Close','Volume','SE','Stock','REFRESH_DATE']]
        
                # additional checking
                latest_data_2=latest_data.loc[latest_data.Date>hist_last_day,:].reset_index(drop=True)
    
                ## Load to temp table 
                latest_data_2.to_sql(name='ALL_STOCK_LATEST', # ALL_STOCK_LATEST
                                     con=engine,
                                     if_exists='replace',
                                     index=False,
                                     chunksize=1000)
                print('ALL_STOCK_LATEST is updated with data as of {:s}, rows updated: {:,.0f}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d'),
                                                                                                       latest_data_2.shape[0])
                     )
    
                cursor.execute("""INSERT INTO ALL_STOCK_HIST 
                                  SELECT *
                                  FROM ALL_STOCK_LATEST 
                                  COMMIT
                                """)
                mydb.commit()
                print('ALL_STOCK_HIST is updated with data as of {:s}'.format(latest_data_2.Date.max().strftime(format='%Y-%m-%d')))
            
            else:
                print('Stock data not available; there are {:,.0f} NA records'.format(latest_data.Open.isnull().sum()))
        
        else:
            print('Not weekday')
    
    else:
        print('ALL_STOCK_HIST is already up-to-date, latest data: {:s}'.format(hist_last_day.strftime(format='%Y-%m-%d')))
        


###############
## Task 
###############
with DAG('Stock_Price_Prediction',
         default_args=yf_args,
         schedule_interval='20 0,2,22 * * *',
         cathup=False
        ) as dag:
    task_yf_base_task=PythonOperator(task_id='Yahoo_Finance_Data_Pull'
                                     python_callable=update_all_stocks_2)

    
task_yf_base_task

# use with: https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html

In [84]:
pre_df=pd.read_sql("SELECT * FROM ALL_STOCK_HIST WHERE DATE>=STR_TO_DATE('2021-08-30','%Y-%m-%d')",con=engine)
print(pre_df.shape)

(6311, 9)


In [85]:
pre_df.head(5)

,Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE
0,2021-08-30,2.680000,2.740000,2.640000,2.700000,45900.0,NASDAQ,AACG,2021-09-01 06:33:59
1,2021-08-30,20.290001,20.299999,19.469999,19.510000,24094300.0,NASDAQ,AAL,2021-09-01 06:33:59
2,2021-08-30,3.860000,3.920000,3.760000,3.870000,33300.0,NASDAQ,AAME,2021-09-01 06:33:59
3,2021-08-30,7.300000,7.360000,7.140000,7.280000,315800.0,NASDAQ,AAOI,2021-09-01 06:33:59
4,2021-08-30,69.089996,69.339996,68.470001,69.199997,162700.0,NASDAQ,AAON,2021-09-01 06:33:59


In [88]:
pre_df.Open.isnull().sum()==0

False

## 02 yahoofinance 
* https://python-yahoofinance.readthedocs.io/en/latest/api.html

# Google Finance 

In [ ]:
from pandas_datareader import data

In [ ]:
pd.__version__